In [1]:
import gym
import numpy as np
import math
import matplotlib.pyplot as plt
env = gym.make('CartPole-v0')
env = gym.make('Acrobot-v1')

# Enviornments

In [ ]:
#print('states high value:')
#print(env.observation_space.high[0]) #4.8  position of cart
#print(env.observation_space.high[1])  #3.4028235e+38   velocity of cart
#print(env.observation_space.high[2])  #0.41887903   angle of pole
#print(env.observation_space.high[3])  #3.4028235e+38  rotation rate of pole

#print('states low value:')
#print(env.observation_space.low[0])   #-4.8
#print(env.observation_space.low[1])  #-3.4028235e+38
#print(env.observation_space.low[2])  #-0.41887903
#print(env.observation_space.low[3])  #-3.4028235e+38
# [position of cart, velocity of cart, angle of pole, rotation rate of pole]

In [ ]:
#print('states high value:')
#print(env.observation_space.high[0]) # 1  cos of first rotational joint 
#print(env.observation_space.high[1])  # 1 sin of first rotational joint
#print(env.observation_space.high[2])  # 1 cos of first rotational joint
#print(env.observation_space.high[3])  # 1 sin of first rotational joint
#print(env.observation_space.high[4])  # 12.566371 angular velocity of 1st joint
#print(env.observation_space.high[5])  # 28.274334 angular velocity of 2nd joint
#[cos(theta1), sin(theta1), cos(theta2), sin(theta2), theta_dot1, theta_dot2]

#Acrobot does not have threshold at which it is considered solved.

"""
    Acrobot is a 2-link pendulum with only the second joint actuated.
    Initially, both links point downwards. The goal is to swing the
    end-effector at a height at least the length of one link above the base.
    Both links can swing freely and can pass by each other, i.e., they don't
    collide when they have the same angle.
    **STATE:**
    The state consists of the sin() and cos() of the two rotational joint
    angles and the joint angular velocities :
    [cos(theta1) sin(theta1) cos(theta2) sin(theta2) thetaDot1 thetaDot2].
    For the first link, an angle of 0 corresponds to the link pointing downwards.
    The angle of the second link is relative to the angle of the first link.
    An angle of 0 corresponds to having the same angle between the two links.
    A state of [1, 0, 1, 0, ..., ...] means that both links point downwards.
    **ACTIONS:**
    The action is either applying +1, 0 or -1 torque on the joint between
    the two pendulum links.
    
    This is an unsolved environment, which means it does not have a specified 
    reward threshold at which it’s considered solved.
"""
"""This environment consists of a two link, two joint robot.
The joint between the links is actuated. At the start of the episode,
the links are hanging downwards. At every timestep the agent chooses
an action that correspond to applying a force to move the actuated
link to the right, to the left, or to not applying a force.
The episode is over once the end of the lower link swings above a certain
height.The goal is to end the episode in the fewest possible timesteps.
We use the OpenAI Gym ‘Acrobot-v1’ environment. This implementation is
based on the system presented in (Geram- ifard et al., 2015).
Each observation is a set consisting of readings from six sensors,
corresponding to the rotational joint angles and velocities of joints and links.
The action space is discrete with three elements, and at each timestep
the environment returns the observation and a reward of -1. 
An episode is terminated after 200 time steps irrespective of
the state of the robot. This is an unsolved environment,
which means it does not have a specified reward threshold at which
it’s considered solved."""

In [2]:
class Qlearning():
    def __init__(self,bins=(1, 1, 6, 12), max_episodes = 1000, max_steps = 195, min_step_size=0.1,min_epsilon=0.0, gamma = 1.0 , decay = 25, action_space = env.action_space.n, env = gym.make('CartPole-v0')):   #max_env_steps=None
        self.bins = bins #(1, 1, 6, 12)--> scale theta and theta' to the intervals (0,6) and (0,12) respectively 
        #rather than 3 buckets for x and xdot, making it 1 makes it so that the state space dimentionalty is signifigantly reduced 
        #Since we assign only one bucket to the cart position and velocity, it means that we ignore these 2 values.
        self.max_episodes = max_episodes # number of training episodes 
        self.max_steps = max_steps # average ticks over 100 episodes required for win
        self.min_step_size = min_step_size  #step size
        self.min_epsilon = min_epsilon #  
        self.gamma = gamma  # discount factor
        self.decay = ada_divisor
        self.action_space = action_space
        self.env = env
        self.QTable = np.zeros((self.buckets + (self.action_space,))) #The Q LookUp Table --> its dimension is  (1 x 1 x 6 x 12 x 2) 
        # or self.QTable = np.empty((self.bins + (self.action_space,)))
        self.actions = None
        self.tau = 500    # tau --> 0 pure exploitation tau --> infinity pure exploration
        self.tau_decay = False
        self.decay_factor_tau = 0.1

Article: "When I was staring at the animation, I noticed that the cart didn’t really move out of bound that often.
Also, I only needed to balance the pole for 200 time steps. It typically didn’t drift that far while balancing the pole.
Therefore, I removed the x and x_dot components and leaving only the theta, and theta_dot components.
With this significantly reduced state-space, I was able to solve the problem much quicker in only 136 episodes !--> why (1,1,...)"
#def __init__(self, actions=None, tau=500, tau_decay=False, decay_factor_tau=0.1):
#assert tau >= 0, 
#assert bool == type(tau_decay),

# Discretize

In [11]:
    def discretize(self, state):
        # observation = obs, reward, done, info = self.env.step(action)--> in training
        # define upper and lower bounds for each state value
        #it is necessary to give finite bounds and reduce the range of the cart velocity and pole velocity at tip – i.e. [-0.5, +0.5] and [-50, +50],
        #respectively – in order to be able to discretise.
        upper_bounds = [env.observation_space.high[0], 0.5, env.observation_space.high[2], math.radians(50)]  
        lower_bounds = [env.observation_space.low[0], -0.5, env.observation_space.low[2], -math.radians(50)]
        descretizedList = list()
        for i in  range(len(state)): #state = tuple (0,0,X,X)--> aka the state space  #scaling factor
            ratios = (state[i] + abs(lower_bounds[i])) / (upper_bounds[i] - lower_bounds[i]) # (state+lower / bin_width) = bucket size
            new_obs = int(round((self.bins[i] - 1) * ratios)) # makes the first two buckets 0, and #bucket_lower := min_x + bucket × bucket size
            new_obs = min(self.bins[i] - 1, max(0, new_obs)) #no matter what the buckets-1 is going to be 0 or greater than 0
            #this makes sure that the first two components of the tuple is 0
            #why are we taking the min between the bucket-1 and the new_obs
            descretizedList.append(new_obs)
        return tuple(descretizedList)

#print(env.observation_space.high[2])  #0.41887903   angle of pole
#print(env.observation_space.high[3])  #3.4028235e+38  rotation rate of pole

I chose those values for [1], and [3] for buckets as I was inspired by a blog post, muetsch and tuzzer--> I tried it with other values and it did not converge
https://ferdinand-muetsch.de/cartpole-with-qlearning-first-experiences-with-openai-gym.html
https://mc.ai/openai-gyms-cart-pole-balancing-using-q-learning/
'''When I was staring at the animation, I noticed that the cart didn’t really move out of bound that often.
Also, I only needed to balance the pole for 200 time steps. It typically didn’t drift that far while balancing the pole.
Therefore, I removed the x and x_dot components and leaving only the theta, and theta_dot components.
With this significantly reduced state-space, I was able to solve the problem much quicker in only 136 episodes... why (1,1,...)
(1, 1, 6, 12)--> number of buckets for each action space rather than 3 buckets for x and xdot, making it 1 makes it so that the state space dimentionalty is signifigantly reduced
I also tried [1] and [3] to be 3.4 as its true value is 3.4028235e+38-> this did not work well however

As states are continuous, discretize them into buckets--> Binning or discretization is the process of transforming numerical variables into categorical counterparts. An example is to bin values for Age into categories such as 20-39, 40-59, and 60-79. Numerical variables are usually discretized in the modeling methods based on frequency tables (e.g., decision trees).

In [ ]:
    def discretize(self, state):
        # observation = obs, reward, done, info = self.env.step(action)--> in training
        # define upper and lower bounds for each state value
        #it is necessary to give finite bounds and reduce the range of the cart velocity and pole velocity at tip – i.e. [-0.5, +0.5] and [-50, +50],
        #respectively – in order to be able to discretise.
        upper_bounds = [1.0, 1.0, 1.0, 1.0, env.observation_space.high[4], env.observation_space.high[5]]  
        lower_bounds = [-1.0,-1.0,-1.0,-1.0, env.observation_space.low[4], env.observation_space.low[5]]
        descretizedList = list()
        for i in  range(len(state)): #state = tuple (0,0,X,X)--> aka the state space  #scaling factor
            ratios = (state[i] + abs(lower_bounds[i])) / (upper_bounds[i] - lower_bounds[i]) # (state+lower / bin_width) = bucket size
            new_obs = int(round((self.bins[i] - 1) * ratios)) # makes the first two buckets 0, and #bucket_lower := min_x + bucket × bucket size
            new_obs = min(self.bins[i] - 1, max(0, new_obs)) #no matter what the buckets-1 is going to be 0 or greater than 0
            #this makes sure that the first two components of the tuple is 0
            #why are we taking the min between the bucket-1 and the new_obs
            descretizedList.append(new_obs)
        return tuple(descretizedList)

# Behavior Policies

## Epsilon-Greedy: Behavior Policy 1

In [4]:
    def action(self, state, epsilon):
        ''' choose an action using the epsilon policy '''
        #exploration-exploitation = np.random.random()
        #best = np.argmax(self.QTable[state])
        #random = env.action_space.sample()  # select a random action (see https://github.com/openai/gym/wiki/CartPole-v0) # exploration
        if np.random.random() <= epsilon:
            action = self.env.action_space.sample()   
        else:
            action = np.argmax(self.QTable[state])  # exploitation
            #print("best state", np.argmax(self.QTable[state]))
        return action

epsilon decay inspired by--> Hands-On Intelligent Agents with OpenAI Gym: Your guide to developing AI using Deep Reinforcment Learning

## For MDP

In [ ]:
    def action(self, state, epsilon):
        if np.random.random() <= epsilon:
            action = random.choice(self.env.actions)   
        else:
            action = np.argmax(self.QTable[state])
        return action

## Uniform Behavior Policy: Behavior Policy 2

In [ ]:
    def action(self, state, epsilon):
        action = self.env.action_space.sample()
        return action

## Boltzmann Exploration Policy: Behavior Policy 3

In [4]:
# https://arxiv.org/pdf/1612.05628.pdf
    def Boltzmann(self, state, tau):
        qList = self.QTable[state] #moves = self.q_matrix[pos_index]
        self.actions = [i for i in range(0, env.action_space.n)]
        # Circumvent math issues with temperature actually being 0
        if self.tau > 0.0001:
            #print(self.tau)
            # Compute action probabilities using tau; when
            # tau is high, we're treating values of very different
            # Q-values as more equally choosable
            action_probs_numes = []
            denom = 0
            for q in qList:
                val = math.exp(q / self.tau)
                action_probs_numes.append(val)
                denom += val  # summation of the values
            action_probs = []
            for x in action_probs_numes:
                probs = x / denom 
                action_probs.append(probs)    
            # Pick random move, in which moves with higher probability are
            # more likely to be chosen, but it is obviously not guaranteed
            prob_sum = 0
            #action = action_probs.index(max(action_probs))
            #print(action)
            #action = np.argmax(action_probs)
            action = random.choices(self.actions, weights=action_probs, k=1)[0] # the weights give a weight to correspond 
            #for each action , and then chooses a random action with each action weighted by it prob. and then chooses
            #I attemped using the action for the max probabilty but it only returned one action each time
        else:
            # Here, we're totally cold; meaning, we're just exploiting
            action = np.argmax(self.QTable[state])
        return action

Explanation: In exploration, we would ideally like to exploit all the information present in the estimated Q-values produced by our network. Boltzmann exploration does just this. Instead of always taking the optimal action, or taking a random action, this approach involves choosing an action with weighted probabilities. To accomplish this we use a softmax over the networks estimates of value for each action. In this case the action which the agent estimates to be optimal is most likely (but is not guaranteed) to be chosen. The biggest advantage over e-greedy is that information about likely value of the other actions can also be taken into consideration. If there are 4 actions available to an agent, in e-greedy the 3 actions estimated to be non-optimal are all considered equally, but in Boltzmann exploration they are weighed by their relative value. This way the agent can ignore actions which it estimates to be largely sub-optimal and give more attention to potentially promising, but not necessarily ideal actions.
Adjusting during training: In practice we utilize an additional temperature parameter (τ) which is annealed over time. This parameter controls the spread of the softmax distribution, such that all actions are considered equally at the start of training, and actions are sparsely distributed by the end of training.

https://github.com/lkgwbr/QLearnGrid/blob/5955dc4241c50896f7ae42b155738b25014f15f3/main.py#L196
https://github.com/lucadivit/Reinforcement_Learning_Maze_Solver/tree/b4c56aec5f6aee56cbf7f5b77d4ec7424f40fb14
Inspired by these codes

# The Update Function

In [12]:
    def Q_Update(self, state, action, reward, new_state, step_size):
        ''' Update the Q-values'''
        # Qk+1(s,a) = Qk(s,a) + alpha[R(s, a, s')+ gamma(maxQk(s,a,))- Qk(s,a))
        self.QTable[state][action] += step_size * (reward + self.gamma * (np.max(self.QTable[new_state])) - self.QTable[state][action])

# Epsilon Decay

### Natural log based decay- decays in begining and is constant all the way through the end

In [2]:
    def decay_epsilon(self, t):  
        ##### DIMINISHING EPSILON ####
        #print(max(self.min_alpha, min(1.0, 1.0 - math.log10((t + 1) / self.ada_divisor))))
        #logarithmic model -->  a + b ln(x)- this is increasing a-blnx- decreasing
        #1- log bc max alpha is 1 and and you have to diminish it from there until you get to the minimum epsilon 
        return max(self.min_epsilon, min(1, 1.0 - math.log((t + 1) / self.decay), math.e))

I attemped several different decay function--> linear, and exponential, and logbase 10, however, I settled on natural log as it worked really well

### constant decay- decay every episode by a factor of 0.99

In [ ]:
    def decay_epsilon(self):
        epsilon = self.epsilon * 0.99
        if epsilon < self.min_epsilon:
            self.epsilon = self.min_epsilon
        else:
            self.epsilon = epsilon
        return epsilon

# Linear Decay

In [ ]:
    def decay_epsilon(self):
        epsilon = self.epsilon - 0.001
        if epsilon < self.min_epsilon:
            self.epsilon = self.min_epsilon
        else:
            self.epsilon = epsilon
        return epsilon

# The Alpha Decay

In [13]:
    def decay_step_size(self, t):
        ##### DIMINISHING STEP SIZES ####
        #print(max(self.min_alpha, min(1.0, 1.0 - math.log10((t + 1) / self.ada_divisor))))
        #logarithmic model -->  a + b ln(x)- this is increasing a-blnx- decreasing
        #1- log bc max alpha is 1 and and you have to diminish it from there until you get to the minimum alpha 
        return max(self.min_step_size, min(1.0, 1.0 - math.log((t + 1) / self.decay), math.e))  #makes sure it doesn't go above 1

I attemped several different decay function--> linear, and exponential, and logbase 10, however, I settled on natural log as it worked really well. In the code for constant alpha I used 0.9. I attemped several different values ranging from 0.1 to 1, however they all preformed relatively similarly--> I am still trying to understand why because it should still work

In [ ]:
    def decay_step_size(self, t):
        if t == 0:
            step_size = 1.0
        elif t > 0:
            step_size = self.alpha / (t ** self.beta)  
        return step_size

This fulfills the requirement of non summable but square summable. 

alpha and beta are the hyperparameters and bounded by the ranges as shown above. 

In [ ]:
    def decay_step_size(self, tt, e, t):
        num = self.alpha
        dem = tt**self.beta
        if e == 0 and t == 0:
            step_size = 1
        elif tt > 0:
            step_size = num / dem
        return step_size

# Moving Average

In [ ]:
    def moving_average(self, rewards, window_size = 10): #A specified window size determines the size of each subset.
        #In statistics, a moving average (rolling average or running average)
        #is a calculation to analyze data points by creating a series of averages of different subsets of the full data set. 
        #The moving average is calculated by adding the rewards over a certain period 
        #and dividing the sum by the total number of periods.
        sum_vec = np.cumsum(np.insert(rewards, 0, 0))  
        moving_ave = (sum_vec[window_size:] - sum_vec[:-window_size]) / window_size  #formula for moving average
        return moving_ave

# The Training

In [14]:
    def learning(self):
        rewards = []
        for e in range(self.max_episodes):  
            # As states are continuous, discretize them into buckets
            state = self.discretize(self.env.reset())  # for each episode the enviornment resets and discretizes the state
            epsilon = self.decay_epsilon(e)
            step_size = self.decay_step_size(e)
            i = 0
            for t in range(self.max_steps):   #for every timestep
                action = self.action(state, epsilon)
                new_state, reward, done, info = self.env.step(action)  #the obeservation space (0,0,x,x)
                new_state = self.discretize(new_state)
                self.Q_Update(state, action, reward, new_state, step_size)
                state = new_state
                i += 1
                if done:
                    break
            rewards.append(i)
        return rewards

# Cartpole

# Acrobot 

In [ ]:
    def learning(self):
        rewards = []
        alphaList = []
        taus = []
        times = 0
        listof50 = deque(maxlen=50)
        #episodes = []
        # this part until the end of the while loop:
        # this is the part of the function that trains the enviornment.
        for e in range(self.max_episodes):  
            # As states are continuous, discretize them into buckets
            # for each episode the enviornment resets and discretizes the state
            state = self.discretize(self.env.reset())
            #epsilon = self.epsilon
            tau = self.decay_tau(e)
            #print("epsilon", epsilon)
            step_size = self.decay_step_size(e)
            #print("alpha", alpha)
            i = 0
            for t in range(self.max_steps):
                #step_size = self.decay_step_size(e)
                #self.env.render()
                if e%2==0:
                    action = self.Boltzmann(state, tau) #, epsi
                elif e%2==1:
                    action = self.action_policy(state, policy)
                new_state, reward, done, info = self.env.step(action)
                #print(reward)
                new_state = self.discretize(new_state)
                self.Q_Update(state, action, reward, new_state, step_size)
                state = new_state
                i += reward  #-1 for each timestep the agent takes to swing up  # reward threshold -100
                if done:
                    break
            stateTuple = list()
            stateList = []
            actionsList = []
            states = []
            for list1 in self.QTable:
                for nextList in list1:
                    for nextList1 in nextList:
                        for nextList2 in nextList1:
                            for nextList3 in nextList2:
                                for nextList4 in nextList3:
                                    actions = np.argmax(nextList4)
                                    actionsList.append(actions)
            observations = np.argwhere(self.QTable >= -600)
            for obsList in observations:
                stateTuple = (obsList[0], obsList[1], obsList[2], obsList[3], obsList[4], obsList[5])
                stateList.append(stateTuple)
            del stateList[1::3]
            del stateList[1::2]
            #print(len(actionsList), len(stateList))
            policy = dict(zip(stateList, actionsList))
            #print(policy)
            if e%2 ==1:
                rewards.append(i)
                alphaList.append(step_size)
                taus.append(tau)
                listof50.append(i)
                mean_listof50 = np.mean(listof50)
                if i >= -100:
                    times+=1
                #print("Solved after " + str(e) + " episodes.")
                #print(str(e) + " episodes: " + str(i) + " reward")
                #return rewards, alphaList, epsilonList
        print(times)
        return rewards, alphaList, taus



# Synchronous

In [ ]:
    def learning(self):
        rewards = []
        alphaList = []
        epsilonList = []
        #episodes = []
        # this part until the end of the while loop:
        # this is the part of the function that trains the enviornment.
        for e in range(self.max_episodes):  
            if e % 10000 == 0:
                print(e)
            #state1 = np.random.randint(0,9) #self.env.reset()
            epsilon = self.decay_epsilon() #e)
            step_size = 0.01 #self.decay_step_size(e)
            for state in self.env.states:
                for action in self.env.actions:
                    transitionVector = self.env.transitions[action][state]
                    new_state = random.choices(population=self.env.states, weights=list(transitionVector))
                    new_state = new_state[0]
                    reward = self.env.rewards[action][state] #[new_state]
                    self.Q_Update(state, action, reward, new_state, step_size)
            rewards.append(np.linalg.norm(self.QTable-self.QFunction))#averageQ)
            alphaList.append(step_size)
            epsilonList.append(epsilon)
            mean_rewards =  np.mean(rewards)
        print(self.QTable)
        #print(self.V)
        return rewards, alphaList, epsilonList

This code only works for the MDP model

# Optimal Policy Extraction

If I starts with this optimal policy would I get the right rewards?

### Cartpole

In [ ]:
            if mean_listof100 >= self.max_steps and e >= 100:
                stateTuple = list()
                stateList = []
                actionsList = []
                states = []
                #print(self.QTable)
                for list1 in self.QTable:
                    for nextList in list1:
                        #print(nextList)
                        #print(np.argwhere(nextList>=0))
                        for nextList1 in nextList:
                            #print(nextList1)
                            #print(np.argwhere(nextList1>=0)) 
                            for nextList2 in nextList1:
                                actions = np.argmax(nextList2)
                                #print(actions)
                                actionsList.append(actions)
                                #print(nextList2)
                observations = np.argwhere(self.QTable>=0)
                j = 0
                for obsList in observations:
                    stateTuple = (obsList[0], obsList[1], obsList[2], obsList[3])
                    stateList.append(stateTuple)
                del stateList[1::2]
                policy = dict(zip(stateList, actionsList))
                print(policy)
                #print("All states:", stateList)
                #print("optimal actions:", actionsList)
                print("Solved after " + str(e) + " episodes.")
                return rewards, alphaList, epsilonList

### Acrobot

In [ ]:
            stateTuple = list()
            stateList = []
            actionsList = []
            states = []
            #print(self.QTable)
            for list1 in self.QTable:
                for nextList in list1:
                    #print(nextList)
                    #print(np.argwhere(nextList>=0))
                    for nextList1 in nextList:
                        #print(nextList1)
                        #print(np.argwhere(nextList1>=0)) 
                        for nextList2 in nextList1:
                            for nextList3 in nextList2:
                                for nextList4 in nextList3:
                                    actions = np.argmax(nextList4)
                                    #print(actions)
                                    actionsList.append(actions)
            observations = np.argwhere(self.QTable>=-500)
            #print(observations)
            j = 0
            for obsList in observations:
                stateTuple = (obsList[0], obsList[1], obsList[2], obsList[3], obsList[4], obsList[5])
                stateList.append(stateTuple)
            del stateList[1::3]
            del stateList[1::2]
            policy = dict(zip(stateList, actionsList))
            #print(policy)
            #print(e, np.argmax(self.QTable))
            listof100.append(i)
            mean_listof100 = np.mean(listof100)

# Trajectories?

The core idea of Model-based RL is using the model and the cost function to locate the optimal path of actions (to be exact — a trajectory of states and actions).

In [ ]:
if __name__ == '__main__':
    ''' Run main program. '''
    #solve = Qlearning()
    #rewards, alpha, epsilon = learning(solve, max_episodes = 500, bins=(1, 1, 6, 12), max_steps = 200,min_alpha=0.1, min_epsilon=0.1, epsilon = 1.0, gamma = 1.0, decay = 25, solved=False, action_space = env.action_space.n)
    trajectory = [] 
    listof_195 = []
    for episode in range(500):
        listof_195.append(195)
    for i in range(100):
        solve = Qlearning()
        rewards, alpha, epsilon = learning(solve)
        trajectory.append(rewards)
        #print(rewards)
        #print("alpha", alpha)
        print(i)
    a = np.array(trajectory)
    final = np.mean(a, axis=0)
    fig = plt.figure(figsize=(16,9))
    plt.plot(final, 
            color = 'b',
            label = 'Expected Rewards')
    plt.plot(listof_195,  # epsilon_hist,
             color='r',
             label='195')
    plt.legend(loc='best')
    plt.title('CartPole-v0: Expected rewards')
    plt.xlabel('episodes')
    plt.ylabel('expected rewards') 
    plt.show()

# The finite and infinite MDPs

## Infinite Horizon MDP

In [ ]:
class MarkovDP:
    def __init__(self,s,a):
        self.num_state             = s
        self.num_action            = a
        self.states                = np.array(range(0,s))
        self.actions               = np.array(range(0,a))
        self.transitions           = np.zeros((a,s,s))
        self.rewards               = np.zeros((a,s,s))
        #print(rewards)
        
# The function below initializes transition probability matrix and rewards marks 

    def initialize_mdp(self):      
        np.random.seed(0)        #for reproducibility 
        self.transitions, self.rewards         = mdptoolbox.example.rand(self.num_state,self.num_action)
        self.rewards                           = np.random.rand(self.num_action, self.num_state) #   ( self.num_state,self.num_state)

This is a MDP created with the mdptoolboox. This is specifically code taken from https://github.com/gt-coar/NesaraREU20/blob/master/Value_Iteration.ipynb

## Finite Horizon MDP

This was inspired with nesara's code as I used her code to create and print the the rewards and transitions for an 11 by 3 mdp, and then manually changed the transitions and rewards to make the 11th state an absorbing state. 

In [ ]:
class Qlearning(MarkovDP):
    def __init__(self, env, max_episodes = 350000, max_steps = 1,min_alpha=0.0, min_epsilon= 0.1, epsilon = 1.00, gamma = 0.95, decay = 25, solved=False):   #max_env_steps=None
        self.max_episodes = max_episodes # number of training episodes 
        self.max_steps = max_steps # average ticks over 100 episodes required for win
        self.alpha = 1.0 # learning rate parameter
        self.beta = 0.5 # learning rate parameter
        self.epsilon = epsilon # exploration rate
        self.min_alpha = min_alpha
        self.min_epsilon = min_epsilon 
        self.solved = solved
        self.gamma = gamma
        self.decay = decay  #works for most numbers between 5 and 35
        self.env = env #gym.make('CartPole-v0')
        self.QTable = np.ones((env.num_state, env.num_action))
        self.zeros = np.full((10,3), 0)
        self.QFunction = np.array([[13.23176522, 13.33925622, 14.05469772], [13.52671959, 13.1224803,  13.22367197], [13.30159516, 13.86810447, 13.51346527], [13.66552722, 13.47151662, 13.23704243], [14.00228974, 13.41027839, 13.44004462], [13.42221627, 13.84567785, 13.56730379], [13.17987508, 13.49701082, 13.59308849], [13.24370111, 13.86450877, 13.89669707], [13.86681663, 13.87073118, 13.73493382], [13.24989143, 13.5198381,  13.37349781]])        

Here the biggest difference from the openAIGym code is how the env is definedit is through initialize mdp) and self.QFunction which is the optimal Q function found through value iteration

In [ ]:
    def learning(self):
        QList = []
        QLog = []
        state = np.random.randint(0, self.env.num_state)
        tt=0
        r = 0
        q = 0
        for e in range(self.max_episodes):  
            if e % 100000 == 0:
                print(e)
            epsilon = self.decay_epsilon() #e)
            step_size = 0.01 #self.decay_step_size(e)
            action = self.action(state, epsilon)
            transitionVector = self.env.transitions[action][state]
            new_state = random.choices(population=self.env.states, weights=list(transitionVector))
            new_state = new_state[0]
            reward = self.env.rewards[action][state]
            self.Q_Update(state, action, reward, new_state, step_size)
            state = new_state
            QList.append(np.linalg.norm(self.QTable - self.QFunction))
            QLog.append(math.log10(np.linalg.norm(self.QTable - self.QFunction)))
        return QList, QLog

This is for the infinite MDP as it is not epsisodic. The new state is chosen based on the state transition probabilities ( the choice is weighted by the probabilities)